# Cleaning Combining Notebook

This notebook takes the pickled outputs from the 3 webscraping cleaning notebooks and combines them into two final dataframes for mean height of men and women by country along with normalized counts of main foods present in recipes. 

### Imports

In [74]:
import pandas as pd
import numpy as np
import re
import pickle

### Combining Dataframes

In [75]:
def combineDF(num_df):
    '''
    This function takes in the number of clean df_country_foods dataframes. 
    It opens the pickle files then appends these dataframes to a list. It then concatenates all dfs into one.
    
    -----
    input: int
    output: DataFrame
    '''
    i = 1
    list_dfs = []
    while i <= num_df:
        with open(('data/df_country_foods_')+str(i)+'.pickle', 'rb') as read_file:
            df_country_foods = pickle.load(read_file)
        list_dfs.append(df_country_foods)
        i += 1
    combine_df = pd.concat(list_dfs)
    return combine_df

In [76]:
combine_df = combineDF(3)

In [153]:
def groupDF(combine_df):
    '''
    This function takes in a clean combined dataframe of foods, food groups, and country names. It returns a 
    transposed dataframe countaining counts of each food mentioned by country
    -----
    input: DataFrame
    output: DataFrame
    '''
    # Grouping into 5 basic food groups
    replace_groups = {'Teas': 'extra','Pulses': 'protein', 'Coffee and coffee products': 'extra', 
                 'Gourds': 'fruits', 'Vegetables': 'vegetables', 'Fruits':'fruits', 
                 'Cereals and cereal products':'grains', 'Milk and milk products':'dairy', 'Baking goods':'extra',
                 'Beverages':'extra', 'Beverages':'extra', 'Aquatic foods':'protein', 'Eggs':'protein', 
                 'Confectioneries':'extra', 'Cocoa and cocoa products':'extra', 'Animal foods':'protein',
                 'Nuts':'protein', 'Snack foods':'extra', 'Soy':'protein'}
    combine_df = combine_df[combine_df.group != 'Herbs and Spices']
    combine_df = combine_df[combine_df.group != 'Herbs and spices']
    combine_df = combine_df[combine_df.group != 'Dishes']
    combine_df = combine_df[combine_df.group != 'Fats and oils']
    combine_df = combine_df.replace({'group':replace_groups})
    combine_df.country = [x.strip() for x in combine_df.country]
    replace_country = {'East Africa':'Ethiopia', 'Africa':'Morocco', 'North Africa':'Morocco', 'Korea':'South Korea',
                      'Islands of the Pacific':'Fiji', 'Virgin Islands':'Puerto Rico', 'Sicily':'Italy',
                      }
    combine_df = combine_df.replace({'country':replace_country})
    combine_df = combine_df.reset_index(drop = True)
    
    # Cleaning countries to reflect same country
    combine_df['country'] = [country.replace('-', ' ') for country in combine_df['country']]
    for i,country in enumerate(combine_df['country']):
        if re.search(r'^United States', country):
            combine_df.loc[i, 'country'] = 'United States'
    for i,country in enumerate(combine_df['country']):
        if re.search(r'^Brazil', country):
            combine_df.loc[i, 'country'] = 'Brazil'
    for i,country in enumerate(combine_df['country']):
        if re.search(r'^Australia', country):
            combine_df.loc[i, 'country'] = 'Australia'
    for i,country in enumerate(combine_df['country']):
        if re.search(r'^Canada', country):
            combine_df.loc[i, 'country'] = 'Canada'
    df_country_groups = combine_df.groupby(['country','group'], as_index = False)['food'].count()
    return df_country_groups

In [154]:
df_country_groups = groupDF(combine_df)

In [166]:
def finalDF(df_country_groups):
    '''
    This function takes in a newly grouped, clean df of country, group, and food count. It transposes the
    group column and adds mean height by country.
    -----
    input: DataFrame
    output: DataFrame, DataFrame
    '''
    # Read in height df
    df_height = pd.read_csv('data/height_country.txt')
    
    # Clean country names in height df to match food group df 
    df_height['Country'] = [country.replace('-', ' ') for country in df_height['Country']]
    for i,country in enumerate(df_height['Country']):
        if re.search(r'^United States', country):
            df_height.loc[i, 'Country'] = 'United States'
    df_height_recent = df_height[df_height['Year of birth'] == 1996] #Use most recent year
    df_height_recent = df_height_recent.reset_index(drop = True)
    df_height_recent = df_height_recent.rename(columns = {'Year of birth':'year', 'Mean height (cm)':'mean', 
                         'Mean height lower 95% uncertainty interval (cm)':'lower95',
                        'Mean height upper 95% uncertainty interval (cm)' : 'upper95',
                                                     'Country':'country'}) #Rename cols 
    df_country_groups['country'] = [x.lower() for x in df_country_groups['country']]
    df_height_recent['country'] = [x.lower() for x in df_height_recent['country']]
    for i,country in enumerate(df_height_recent['country']):
        if re.search(r'^vie', country):
            df_height_recent.loc[i, 'country'] = 'vietnam'
    
    # Combine height df and food df
    final_df = df_country_groups.set_index('country').join(df_height_recent.set_index('country'), how = 'left')
    combined_men_group = final_df[final_df['Sex'] == 'Men'] #Split men and women
    combined_women_group = final_df[final_df['Sex'] == 'Women']
    cols = ['ISO', 'Sex', 'year'] #Drop unnecessary cols
    combined_men_group = combined_men_group.drop(cols, axis = 1)
    combined_women_group = combined_women_group.drop(cols, axis = 1)
    
    # Transpose df to have food groups as columns
    food_men = combined_men_group[['food', 'group']]
    food_women = combined_women_group[['food', 'group']]
    food_men = food_men.pivot(columns = 'group', values = 'food')
    food_women = food_women.pivot(columns = 'group', values = 'food')
    food_men = food_men.replace(np.nan,0)
    food_women = food_women.replace(np.nan,0)
    y_men = combined_men_group.groupby('country')['mean'].mean()
    y_women = combined_women_group.groupby('country')['mean'].mean()
    combined_men_group = food_men.join(y_men, how = 'left')
    combined_women_group = food_women.join(y_women, how = 'left')
    return combined_men_group, combined_women_group

In [167]:
df_country_groups.country.unique()

array(['algeria', 'argentina', 'armenia', 'australia', 'austria',
       'bangladesh', 'belgium', 'brazil', 'c te d ivoire', 'cameroon',
       'canada', 'chile', 'china', 'costa rica', 'cuba', 'czech republic',
       'denmark', 'egypt', 'england', 'ethiopia', 'finland', 'france',
       'germany', 'ghana', 'greece', 'guatemala', 'haiti', 'hungary',
       'india', 'indonesia', 'iran', 'iraq', 'ireland',
       'islands of the pacific', 'israel', 'italy', 'jamaica', 'japan',
       'kazakhstan', 'kenya', 'lebanon', 'liberia', 'malaysia', 'mexico',
       'morocco', 'mozambique', 'netherlands', 'nigeria', 'norway',
       'pakistan', 'persia', 'peru', 'philippines', 'poland', 'portugal',
       'puerto rico', 'russia', 'saudi arabia', 'scotland', 'slovenia',
       'south africa', 'south korea', 'spain', 'sweden', 'tanzania',
       'thailand', 'turkey', 'ukraine', 'united kingdom', 'united states',
       'vietnam', 'zimbabwe'], dtype=object)

In [168]:
final_df_men, final_df_women = finalDF(df_country_groups)

In [165]:
with open('data/final_df_men.pickle', 'wb') as to_write:
    pickle.dump(final_df_men, to_write)
with open('data/final_df_women.pickle', 'wb') as to_write:
    pickle.dump(final_df_women, to_write)